In [48]:
# # Required environment: Python 3.10 or more

# # 🧰 Install required packages (run this cell only once per environment setup)
# # Install the core IBL tools: ONE-api (data access), ibllib (IBL tools),
# !pip install ONE-api ibllib 

# # Install brainbox (analysis utilities)
# !pip install git+https://github.com/int-brain-lab/ibllib.git

# # Install the scientific and plotting libraries
# !pip install matplotlib seaborn pandas numpy scipy

In [65]:
import gc
import numpy as np
import scipy.io as sio  # For saving

# For clearing cache
import shutil
import os


from one.api import ONE
from brainbox.io.one import SpikeSortingLoader, SessionLoader
from brainbox.behavior.training import compute_performance
from brainbox.population.decode import get_spike_counts_in_bins

# Connect to IBL data server
one = ONE(base_url='https://openalyx.internationalbrainlab.org')
print(one.cache_dir)

# =========================================
# 🔁 Loop through contrast × prior settings
# =========================================
# Spike count window settings
window_start = 0          # in seconds
window_end = 300 / 1000   # in seconds

# nNeurons_lb = 10
nTrials_lb = 2 # Minimum number of trials
RT_ub = 2000 # trials with RT more than this dur (in ms) will be removed

# Identify eid (id for experiment)
eid = '28741f91-c837-4147-939e-918d38d849f2' # Churchlandlab
label = 'probe00'
target_region = 'CP'
flag_good_quality = False # False: do no do quality control
prior = .5 # no bias

/Users/xueshutian/Downloads/ONE/openalyx.internationalbrainlab.org


In [66]:
# --- Load session ---
session_info = one.get_details(eid)

# Identify the corresponding eid (id for experiment)
pids, labels = one.eid2pid(eid)

for pid, pname in zip(pids, labels):
  print(f'pid: {pid}, labels: {pname}')

trials = one.load_object(eid, 'trials', collection='alf')


pid: bf591043-03c2-48bb-9197-e17e85aaeb8f, labels: probe00
pid: 5458cb27-d065-4626-bcd8-1aa775e1115e, labels: probe01


In [67]:
# ================================
# Load trial and probe info
# ================================
pids, labels = one.eid2pid(eid)
trials = one.load_object(eid, 'trials', collection='alf')
contrast_levels = np.unique(trials['contrastLeft'])
prior_levels = np.unique(trials['probabilityLeft'])

# assert that prior_levels has three levels: 0.2, 0.5 and 0.8
assert np.array_equal(prior_levels, [0.2, 0.5, 0.8]), f"Unexpected prior_levels: {prior_levels}"

# ================================
# Load spikes and clusters
# ================================
ssl = SpikeSortingLoader(eid=eid, one=one, pname=label)
spikes, clusters, channels = ssl.load_spike_sorting()
clusters = ssl.merge_clusters(spikes, clusters, channels)

print(' ==== SESSION LOADED ====')


 ==== SESSION LOADED ====


## Save respNeural and respBehav to .mat files


In [68]:
# Preallocate 
respNeural = []  # List to store spike counts
respBehav = []  # List to store behavioral data

# =========================
# 🔁 Loop over contrasts
# =========================
for contrast in contrast_levels:
    
    # -------------------------
    # 🧪 Trial selection
    # -------------------------
    trial_mask = ((trials['contrastLeft'] == contrast) | (trials['contrastRight'] == contrast)) & \
                (trials['probabilityLeft'] == prior) 
    valid_trial_times = trials['stimOn_times'][trial_mask]

    # if len(valid_trial_times) < nTrials_lb:
    #     continue

    # -------------------------
    # ⏱️ Define time bins per trial
    # -------------------------
    time_bins = np.stack([
        valid_trial_times + window_start,
        valid_trial_times + window_end
    ], axis=1)

    # -------------------------
    # 🔍 Spike filtering
    # -------------------------
    is_valid_spike = np.isin(spikes['clusters'], valid_cluster_ids)
    spike_times = spikes['times'][is_valid_spike]
    spike_clusters = spikes['clusters'][is_valid_spike]

    # -------------------------
    # 📊 Spike count matrix
    # -------------------------
    spike_counts, used_cluster_ids = get_spike_counts_in_bins(
        spike_times, spike_clusters, time_bins
    )  # Output: (n_trials x n_neurons)
    
    # -------------------------
    # 🎯 Behavioral data
    # -------------------------

    choice = trials['choice'][trial_mask]
    contrast_row = np.full_like(choice, contrast, dtype=float)
    stimLeft = trials['contrastLeft'][trial_mask] # not nan is stim is on the left; nan is stim is on the right
    stim_row = np.where(~np.isnan(stimLeft), -1, 1)  # -1 for left, 1 for right
    print(stim_row)
    behav_matrix = np.vstack([contrast_row, choice, stim_row])  # Shape: (2 x n_trials)

    # -------------------------
    # 💾 Save to list
    # -------------------------
    respNeural.append(spike_counts)
    respBehav.append(behav_matrix)

    print(f"  ✅ Stored contrast {contrast} | Trials: {len(choice)}")

[-1  1 -1 -1  1  1 -1 -1  1  1]
  ✅ Stored contrast 0.0 | Trials: 10
[ 1 -1  1  1  1  1 -1 -1 -1 -1 -1  1  1 -1 -1 -1  1  1  1 -1]
  ✅ Stored contrast 0.0625 | Trials: 20
[ 1  1  1 -1  1  1  1  1 -1 -1 -1 -1 -1 -1  1  1  1 -1 -1 -1]
  ✅ Stored contrast 0.125 | Trials: 20
[-1  1 -1  1  1 -1 -1  1 -1 -1  1 -1  1  1  1  1 -1 -1 -1  1]
  ✅ Stored contrast 0.25 | Trials: 20
[-1  1  1 -1 -1 -1  1  1  1 -1 -1 -1  1 -1 -1  1  1  1  1 -1]
  ✅ Stored contrast 1.0 | Trials: 20
[]
  ✅ Stored contrast nan | Trials: 0


In [ ]:
# ------------------------------------------
# 🎯 Neuron Selection by Region and Quality
# ------------------------------------------
region_mask = clusters['acronym'] == target_region
n_neurons = len(clusters['acronym'])
quality_mask = clusters['label'] == 1 if flag_good_quality else np.ones(n_neurons, dtype=bool)
valid_cluster_mask = region_mask & quality_mask
valid_cluster_ids = clusters['cluster_id'][valid_cluster_mask]

print(f"✅ Selected {len(valid_cluster_ids)} neurons in region {target_region} with quality={flag_good_quality}")

# ------------------------------------------
# 🗃️ Preallocate storage
# ------------------------------------------
respNeural = []
respBehav = []

# ------------------------------------------
# 🔁 Loop over contrasts
# ------------------------------------------
for contrast in contrast_levels:

    # -------------------------
    # 🧪 Trial selection (initial mask)
    # -------------------------
    trial_mask = ((trials['contrastLeft'] == contrast) | (trials['contrastRight'] == contrast)) & \
                 (trials['probabilityLeft'] == prior)

    # Reaction time filtering
    rt = trials['response_times'] - trials['stimOn_times']
    trial_mask &= (rt <= RT_ub) & ~np.isnan(rt)

    # Early movement exclusion
    movement_latency = trials['firstMovement_times'] - trials['stimOn_times']
    no_early_movement_mask = (movement_latency >= window_end) | np.isnan(movement_latency)
    trial_mask &= no_early_movement_mask

    # Get final valid trial times
    valid_trial_times = trials['stimOn_times'][trial_mask]

    # if len(valid_trial_times) < nTrials_lb:
    #     print(f"⚠️ Skipping contrast {contrast} due to insufficient trials ({len(valid_trial_times)})")
    #     continue

    # -------------------------
    # ⏱️ Define time bins
    # -------------------------
    time_bins = np.stack([
        valid_trial_times + window_start,
        valid_trial_times + window_end
    ], axis=1)

    # -------------------------
    # 🔍 Filter spikes
    # -------------------------
    is_valid_spike = np.isin(spikes['clusters'], valid_cluster_ids)
    spike_times = spikes['times'][is_valid_spike]
    spike_clusters = spikes['clusters'][is_valid_spike]

    # -------------------------
    # 📊 Spike count matrix
    # -------------------------
    spike_counts, used_cluster_ids = get_spike_counts_in_bins(
        spike_times, spike_clusters, time_bins
    )  # shape: (n_trials, n_neurons)

    # -------------------------
    # 🎯 Behavioral data
    # -------------------------
    choice = trials['choice'][trial_mask]
    contrast_row = np.full_like(choice, contrast, dtype=float)
    stimLeft = trials['contrastLeft'][trial_mask]
    stim_row = np.where(~np.isnan(stimLeft), -1, 1)
    print(stim_row)
    behav_matrix = np.vstack([contrast_row, choice, stim_row])  # shape: (3, n_trials)

    # -------------------------
    # 💾 Save results
    # -------------------------
    respNeural.append(spike_counts)
    respBehav.append(behav_matrix)

    print(f"  ✅ Stored contrast {contrast:.2f} | Trials: {len(choice)}")

print('===== ✅ ALL DONE =====')

✅ Selected 369 neurons in region CP with quality=False
[-1  1 -1 -1  1  1 -1 -1  1  1]
  ✅ Stored contrast 0.00 | Trials: 10


KeyboardInterrupt: 

In [53]:
import scipy.io as sio
import numpy as np

# Convert list of arrays to dtype=object arrays so MATLAB sees them as cells
respNeural_cell = np.empty(len(respNeural), dtype=object)
respBehav_cell = np.empty(len(respBehav), dtype=object)

for i in range(len(respNeural)):
    respNeural_cell[i] = respNeural[i]
    respBehav_cell[i] = respBehav[i]

# Define file name
filename = f"data_{target_region}_bias{prior}_QC{flag_good_quality}.mat"

# Prepare dictionary for saving
save_dict = {
    'respNeural': respNeural_cell,
    'respBehav': respBehav_cell,
    'contrast_levels': np.array(contrast_levels),
    'prior': prior,
    'region': target_region,
    'good_quality': flag_good_quality
}

# Save to .mat file
sio.savemat(filename, save_dict)

print(f"✅ Saved to {filename}")

✅ Saved to data_CP_bias0.5_QCTrue.mat
